In [1]:
import numpy as np
import pandas as pd


In [2]:
train = pd.read_csv("https://bit.ly/fc-ml-titanic")

In [4]:
# csv 파일의 dataset 가장 앞의 5개를 보여줌 (default 대신 정수를 넣으면 개수만큼 보여줌)
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* passengerId : boarding ID num
* survived : 1=live, 0=death
* Pclass : cabin level
* Name
* Sex : female, male
* SibSp : bro, sis, wife/husband num
* Parch : parents, son/dauther num
* Fare, cabin Num
* Embarked : on board port

### dividing train / vaildation dataset  (전처리)
1. define btween feature and label
2. dividing feature and label

In [5]:
# 학습에 필요한
# X로 사용할 값 (제공)
feature = ["Pclass", "Sex", "Age", "Fare"]
# Y로 사용할 값 (정답, 예측/비교값)
label = ["Survived"]


In [7]:
# X, Y에 대해 train 할 값의 일부를 확인 (제대로 값을 가지고 오는지 확인)
train[feature].head()
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [8]:
# 학습전 결손 데이터가 있는지에 대해서 확인해서 가공해줘야함
# 기존 자료가 왜곡되지 않도록 함
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
# 데이터셋 나누기
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train[feature], train[label], test_size=0.2, random_state=123)

In [13]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(712, 4)
(712, 1)
(179, 4)
(179, 1)


## Preprocess (전처리) : Missing value(결측치)를 처리

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [15]:
# pandas의 isnull method 로 count
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [17]:
# 특정 칼럼값으로 한정할때
train["Age"].isnull().sum()

177

### Processing int type missing value
#### normally use average value

In [18]:
# o으로 채우기, 1개 일때 fillna()
# describe() 통계치 데이터를 보여줌
# 넣고 리턴처리 한게 아니라서 실제 업데이트가 된게 아닌 상태
train["Age"].fillna(0).describe()


count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [19]:
# 평균으로 채우기
# describe() 통계치 데이터를 보여줌

# meanAge = train["Age"].mean()
# print(meanAge)
# train["Age"].fillna(meanAge).describe()

train["Age"].fillna(train["Age"].mean()).describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

#### imputer : from scikit-learn, more than 2 column be filled
##### but, data must be number type (int, float)

In [20]:
from sklearn.impute import SimpleImputer

# define imputer model
imputer = SimpleImputer(strategy="mean")


In [21]:
# Fit
imputer.fit(train[["Age","Pclass"]])

SimpleImputer()

In [23]:
# Transform
result = imputer.transform(train[["Age","Pclass"]])
print(result)

[[22.          3.        ]
 [38.          1.        ]
 [26.          3.        ]
 ...
 [29.69911765  3.        ]
 [26.          1.        ]
 [32.          3.        ]]


In [25]:
# imputed data save
train[["Age","Pclass"]] = result

In [26]:
train[["Age","Pclass"]].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [27]:
# after update null valuse, recheck well-updated
train[["Age","Pclass"]].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


In [29]:
# Roll back to original (reset imputer)

train = pd.read_csv("https://bit.ly/fc-ml-titanic")

In [30]:
# recheck reset
train[["Age","Pclass"]].isnull().sum()

Age       177
Pclass      0
dtype: int64

In [31]:
# define imputer model
imputer = SimpleImputer(strategy="mean")

In [33]:
# fit_trasform()        cf.fit() + transform()
result = imputer.fit_transform(train[["Age","Pclass"]])

In [34]:
train[["Age","Pclass"]] = result

In [35]:
train[["Age","Pclass"]].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [36]:
train[["Age","Pclass"]].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


### Processing categorical missing values

In [37]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [38]:
# Embarked
train["Embarked"].fillna("S").describe()

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [39]:
imputer = SimpleImputer(strategy="most_frequent")

In [40]:
result = imputer.fit_transform(train[["Embarked","Cabin"]])

In [41]:
train[["Embarked","Cabin"]] = result

In [42]:
train[["Embarked","Cabin"]].isnull().sum()

Embarked    0
Cabin       0
dtype: int64

In [ ]:
### Label encoding